# Tests on some real data
This notebook was used just to test the model on some data captured with an Android smartphone using the app https://play.google.com/store/apps/details?id=de.lorenz_fenster.sensorstreamgps&hl=en

In [153]:
# generic imports
import pandas as pd
import numpy as np
import time
from keras.models import model_from_json
import numpy

def ohe_to_label(ohe_labels):
    Y = [np.argmax(t) for t in ohe_labels]
    return Y

In [154]:
# mean and std used during training
mean = [-8.33711405e+00, 6.74759490e-01, 7.00107768e-01, -1.04606607e-02, 6.75372537e-03, 2.03051039e-02]
std = [4.27451193, 3.23502816, 3.52895212, 0.53593171, 0.42801951, 0.35004746]

In [155]:
class_conversion = {
    '0': 'falling',
    '1': 'jumping',
    '2': 'lying',
    '3': 'running',
    '4': 'sitting',
    '5': 'standing',
    '6': 'walking'
}

# Tests on simulated real time data from training set
To use the dataset `df.csv` you should export the DataFrame from `001-data-exploration.ipynb`. The dataset is the original one, NOT normalized, without the transients.

In [156]:
# read the original Pandas dataframe
df = pd.read_csv('df.csv')
df = df.drop('Unnamed: 0', axis=1)

In [158]:
df.head()

,acc_x,acc_y,acc_z,gyr_x,gyr_y,gyr_z,labels
0,-9.430002,1.973394,-1.573587,-0.053005,0.045182,0.035234,standing
1,-9.426304,1.966108,-1.573818,-0.055009,0.052240,0.061471,standing
2,-9.433605,1.967885,-1.577345,-0.034780,0.030520,0.050830,standing
3,-9.433682,1.956968,-1.603091,-0.060378,0.067733,0.059497,standing
4,-9.437164,1.944125,-1.599782,-0.074121,0.052016,0.067737,standing


In [159]:
df.shape

(1827646, 7)

In [160]:
# normalize acceleration and gyroscope
columns_to_norm = [
    'acc_x', 'acc_y', 'acc_z',
    'gyr_x', 'gyr_y', 'gyr_z']
df[columns_to_norm] = (df[columns_to_norm]-mean)/std

In [161]:
# load model
json_file = open('m_1d2d_01_reg-augmented-model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
# load weights into new model
loaded_model.load_weights("m_1d2d_01_reg-augmented-weights.h5")
print("Loaded model from disk")
loaded_model.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])

Loaded model from disk


In [164]:
# just testing how much time does it take to do a prediction over 50 samples
df = df.drop('labels', axis=1)
for i in range(50):
    X_test = df.iloc[128*i:(128*i)+128]
    X_test_arr = np.array(X_test).reshape(1,1,128,6)
    start = time.time()
    scores = loaded_model.predict(X_test_arr)
    end = time.time()
    print(end-start)
    print(scores)

0.34441351890563965
[[  5.98257566e-06   3.08926283e-05   4.32323674e-07   1.18596347e-06
    2.71066837e-03   9.97245431e-01   5.45205148e-06]]
0.030178070068359375
[[  1.99623773e-05   1.34613438e-04   2.00302748e-06   4.46931836e-06
    3.88201349e-03   9.95932162e-01   2.47782536e-05]]
0.01922440528869629
[[  5.69179538e-05   1.00151192e-04   1.22004349e-05   2.29002635e-05
    2.77114920e-02   9.71991003e-01   1.05380095e-04]]
0.044544219970703125
[[  4.35945840e-05   2.51219055e-04   9.00140003e-06   6.51908476e-06
    4.17627720e-03   9.94136214e-01   1.37722550e-03]]
0.08860492706298828
[[  7.17001831e-06   2.57227657e-04   6.31896455e-07   8.29035798e-07
    1.11948024e-03   9.98593032e-01   2.16351291e-05]]
0.035262107849121094
[[  1.15936132e-07   2.74890203e-06   5.34952438e-09   1.08083977e-08
    1.26842322e-04   9.99869227e-01   1.04009200e-06]]
0.03746771812438965
[[  1.89902043e-04   8.64676607e-04   5.19265268e-05   7.47439699e-05
    4.92417440e-02   9.49298918e-01  

Each prediction takes less then 0.01 seconds to be computed, there should be no problem in doing predictions in real time

# Real sensor test
Doing some tests with the data captured by the aforementioned Android app. The data was saved in a .csv file. Unfortunately when the phone screen went in standby the app stopped saving data, so there are a lot of gaps. Using the initial part of the datasets however is still doable and gives a sense of the usability of the model.

In [165]:
# swapping x and y axis to match the axis used during training
cols = ['acc_y', 'acc_x', 'acc_z', 'gyr_y', 'gyr_x', 'gyr_z']

usecols = [2,3,4,6,7,8]
realdf = pd.read_csv('./test-signals/Jump.csv', names=cols, usecols=usecols)
# invert x axis
realdf['acc_x'] *= -1

In [166]:
realdf.head()

,acc_y,acc_x,acc_z,gyr_y,gyr_x,gyr_z
0,0.335,-10.126,-0.489,0.003,-0.026,0.005
1,0.254,-10.121,-0.580,-0.014,0.017,-0.008
2,0.182,-10.131,-0.747,-0.006,0.014,-0.029
3,0.273,-10.035,-0.374,-0.036,0.025,-0.046
4,0.489,-10.088,-0.388,-0.061,0.032,-0.054


In [167]:
# reordering the columns of the dataset
realdf = realdf.reindex_axis(sorted(realdf.columns), axis=1)
realdf.head()

,acc_x,acc_y,acc_z,gyr_x,gyr_y,gyr_z
0,-10.126,0.335,-0.489,-0.026,0.003,0.005
1,-10.121,0.254,-0.580,0.017,-0.014,-0.008
2,-10.131,0.182,-0.747,0.014,-0.006,-0.029
3,-10.035,0.273,-0.374,0.025,-0.036,-0.046
4,-10.088,0.489,-0.388,0.032,-0.061,-0.054


In [168]:
# normalizing the dataset
realdf_norm = (realdf-mean)/std
realdf_norm.mean()

acc_x   -0.438589
acc_y   -0.117227
acc_z   -0.316258
gyr_x    0.189735
gyr_y   -0.032092
gyr_z   -0.077734
dtype: float64

In [169]:
realdf_norm.shape

(1240, 6)

In [171]:
# checking the first 15 samples
for i in range(15):
    X_test = realdf_norm.iloc[64*i:(64*i)+128]
    X_test_arr = np.array(X_test).reshape(1,1,128,6)
    start = time.time()
    scores = loaded_model.predict(X_test_arr)
    end = time.time()
    print(scores)
    activity_id = str(ohe_to_label(scores)[0])
    print(class_conversion[activity_id])

[[  5.50034528e-23   1.00000000e+00   1.03761648e-30   3.23431497e-29
    1.92791624e-32   3.25606082e-17   1.33126384e-23]]
jumping
[[  2.46141723e-31   1.00000000e+00   0.00000000e+00   2.20345664e-38
    0.00000000e+00   6.71334066e-26   7.93265059e-31]]
jumping
[[  1.62968712e-30   1.00000000e+00   0.00000000e+00   3.61639238e-37
    0.00000000e+00   7.63332930e-30   1.27069235e-32]]
jumping
[[  4.91270182e-29   1.00000000e+00   0.00000000e+00   3.86697148e-35
    0.00000000e+00   9.43191510e-29   1.26629029e-30]]
jumping
[[  1.24372403e-28   1.00000000e+00   1.36114005e-38   2.42877773e-34
    0.00000000e+00   4.97496301e-26   4.44467542e-30]]
jumping
[[  3.22445337e-33   1.00000000e+00   0.00000000e+00   0.00000000e+00
    0.00000000e+00   1.84383930e-24   6.15397649e-34]]
jumping
[[  7.92438119e-34   1.00000000e+00   0.00000000e+00   0.00000000e+00
    0.00000000e+00   1.26888055e-25   1.65966872e-31]]
jumping
[[  8.36170917e-34   1.00000000e+00   0.00000000e+00   0.00000000e+00

In the above example, jumping signals have been predicted with the correct label.